Hi everyone! This is the notebook we will use tf.Keras to train/evaluate/predict a classifer for cat/dog/bird with convolutional layers. With tf.Keras things become very simple all we have to do is to combine codes from previous notebooks!

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import utils
import os

print(tf.__version__)

In [ ]:
class_names = ['bird','cat','dog']

Preparing Data

In [ ]:
filenames = []
labels = []
for filename in os.listdir("../data"):
    if filename.startswith("bird"):
        labels.append(0)
        filenames.append(os.path.join("../data",filename))
    elif filename.startswith("cat"):
        labels.append(1)
        filenames.append(os.path.join("../data",filename))
    elif filename.startswith("dog"):
        labels.append(2)
        filenames.append(os.path.join("../data",filename))
print(filenames)
print(labels)

In [ ]:
path_dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

def load_and_preprocess_image(filename, label):
    raw_image = tf.io.read_file(filename)
    image_tensor = tf.image.decode_jpeg(raw_image)
    image_tensor = tf.image.resize(image_tensor, [224, 224])
    image_tensor /= 255.0
    return image_tensor,label
imageDataset = path_dataset.map(load_and_preprocess_image).shuffle(buffer_size=100)

In [ ]:
trainDataset = imageDataset.take(int(len(filenames)* 0.7))
testDataset = imageDataset.skip(int(len(filenames)* 0.7))

Preparing Model

In [ ]:
cnnModel = keras.Sequential()
cnnModel.add(layers.Conv2D(input_shape=[224,224,3],filters = 32, kernel_size = 4, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 32, kernel_size = 4, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 4, strides = 4))
cnnModel.add(layers.Dropout(0.3))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 4, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 4, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 4, strides = 4))
cnnModel.add(layers.Dropout(0.3))
cnnModel.add(layers.Conv2D(filters = 128, kernel_size = 4, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 128, kernel_size = 4, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Dropout(0.3))
cnnModel.add(layers.Flatten())
cnnModel.add(layers.Dense(units = 512, activation = 'relu'))
cnnModel.add(layers.Dense(units = 3, activation = 'softmax'))
cnnModel.summary()

Compiling Model

In [ ]:
cnnModel.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0005),
             loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Training Procedure

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]
history = cnnModel.fit(trainDataset.repeat().batch(5), epochs=20, steps_per_epoch=30, validation_data=testDataset.batch(3), callbacks = callbacks)

In [ ]:
# history record the accuracy/loss on training set/validation set after every epoch 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'valivation'], loc='upper left')
plt.show()

You must set the batch_size for the dataset before sending it to the model for training/evaluation.

In [ ]:
result = cnnModel.evaluate(testDataset.batch(10))

In [ ]:
result

Prediction Phase

In [ ]:
predict_result = tf.argmax(cnnModel.predict(testDataset.take(9).batch(9)),axis=1)

In [ ]:
predict_result.numpy()

In [ ]:
utils.show_images(testDataset, class_names, predict_result)